<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

In [3]:
hist_trans = feather.read_dataframe('hist_trans')
new_hist_trans = feather.read_dataframe('new_hist_trans')

In [4]:
# DataFrameSummary(new_hist_trans).summary().T

In [6]:
 import featuretools as ft

In [7]:
PATH = 'data/elo/'

In [8]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

In [9]:
train.head().T

,0,1,2,3,4
first_active_month,2017-06,2017-01,2016-08,2017-09,2017-11
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749


In [24]:
hist_trans.head().T

,0,1,2,3,4
authorized_flag,1,1,1,1,1
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,0,0,0,0,0
installments,0,0,0,0,0
category_3,A,A,A,A,A
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11
purchase_amount,-0.703331,-0.733128,-0.720386,-0.735352,-0.722865


In [28]:
temp = hist_trans.loc[hist_trans['card_id'].isin(['C_ID_92a2005557'])]

In [32]:
temp.columns.values

array(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2',
       'state_id', 'subsector_id', 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day',
       'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Is_month_end', 'purchase_Is_month_start',
       'purchase_Is_quarter_end', 'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start', 'purchase_Elapsed', 'purchased_on_weekend', 'month_diff'], dtype=object)

In [33]:
temp.drop(['purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day',
       'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Is_month_end', 'purchase_Is_month_start',
       'purchase_Is_quarter_end', 'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start', 'purchase_Elapsed'], axis=1, inplace=True)

In [35]:
temp.columns.values

array(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2',
       'state_id', 'subsector_id', 'purchased_on_weekend', 'month_diff'], dtype=object)

In [36]:
cat_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1', 'category_3',
       'merchant_category_id', 'merchant_id', 'category_2',
       'state_id', 'subsector_id', 'purchased_on_weekend', 'month_diff']

In [38]:
cat_dict = {c:ft.variable_types.Categorical for c in cat_cols}

In [45]:
cat_dict

{'authorized_flag': featuretools.variable_types.variable.Categorical,
 'card_id': featuretools.variable_types.variable.Categorical,
 'city_id': featuretools.variable_types.variable.Categorical,
 'category_1': featuretools.variable_types.variable.Categorical,
 'category_3': featuretools.variable_types.variable.Categorical,
 'merchant_category_id': featuretools.variable_types.variable.Categorical,
 'merchant_id': featuretools.variable_types.variable.Categorical,
 'category_2': featuretools.variable_types.variable.Categorical,
 'state_id': featuretools.variable_types.variable.Categorical,
 'subsector_id': featuretools.variable_types.variable.Categorical,
 'purchased_on_weekend': featuretools.variable_types.variable.Categorical,
 'month_diff': featuretools.variable_types.variable.Categorical,
 'index': featuretools.variable_types.variable.Index}

In [40]:
es = ft.EntitySet()

In [41]:
new_trans_es = es.entity_from_dataframe('new_trans', temp, 'index', time_index='purchase_date',
                                        variable_types=cat_dict)

2019-01-19 21:48:42,534 featuretools.entityset - WARNING    index index not found in dataframe, creating new integer column


In [42]:
new_trans_es

Entityset: None
  Entities:
    new_trans [Rows: 260, Columns: 17]
  Relationships:
    No relationships

In [43]:
new_trans_es['new_trans']

Entity: new_trans
  Variables:
    index (dtype: index)
    installments (dtype: numeric)
    month_lag (dtype: numeric)
    purchase_amount (dtype: numeric)
    purchase_date (dtype: datetime_time_index)
    authorized_flag (dtype: categorical)
    card_id (dtype: categorical)
    city_id (dtype: categorical)
    category_1 (dtype: categorical)
    category_3 (dtype: categorical)
    merchant_category_id (dtype: categorical)
    merchant_id (dtype: categorical)
    category_2 (dtype: categorical)
    state_id (dtype: categorical)
    subsector_id (dtype: categorical)
    purchased_on_weekend (dtype: categorical)
    month_diff (dtype: categorical)
  Shape:
    (Rows: 260, Columns: 17)

In [44]:
train_temp = train[train['card_id'].isin(['C_ID_92a2005557'])]

In [46]:
train_temp.columns.values

array(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'], dtype=object)

In [47]:
train_cat_cols = ['card_id', 'feature_1', 'feature_2', 'feature_3']

In [48]:
train_cat_cols = {c:ft.variable_types.Categorical for c in train_cat_cols}

In [50]:
new_trans_es = es.entity_from_dataframe('train', train_temp, 'card_id', time_index='first_active_month',
                                        variable_types=train_cat_cols)

In [51]:
new_trans_es

Entityset: None
  Entities:
    new_trans [Rows: 260, Columns: 17]
    train [Rows: 1, Columns: 6]
  Relationships:
    No relationships

In [52]:
new_r = ft.Relationship( new_trans_es['train']['card_id'],new_trans_es['new_trans']['card_id'])

In [53]:
new_trans_es = new_trans_es.add_relationship(new_r)

In [54]:
new_trans_es

Entityset: None
  Entities:
    new_trans [Rows: 260, Columns: 17]
    train [Rows: 1, Columns: 6]
  Relationships:
    new_trans.card_id -> train.card_id

In [58]:
fm, fd = ft.dfs(entityset=new_trans_es, target_entity='train')

In [60]:
fm.T

card_id,C_ID_92a2005557
target,-0.820283
feature_1,5
feature_2,2
feature_3,1
SUM(new_trans.installments),4
SUM(new_trans.month_lag),-1017
SUM(new_trans.purchase_amount),-165.969
STD(new_trans.installments),0.123314
STD(new_trans.month_lag),2.39769
STD(new_trans.purchase_amount),0.212139


In [61]:
# DataFrameSummary(temp).summary().T

In [64]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 
                   'subsector_id']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min', np.ptp]
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    cols_vals = df.columns.values
    for col in ['category_2','category_3', 'authorized_flag']:
        if col in cols_vals:
            df[col+'_amount_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
            df[col+'_amount_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
            aggs[col+'_amount_mean'] = ['mean']
            aggs[col+'_amount_sum'] = ['sum']

    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')
    new_df["purchase_date_ptp"] = new_df["purchase_date_ptp"].dt.days
    new_df['purchase_date_diff'] = (new_df['purchase_date_max'] - new_df['purchase_date_min']).dt.days
    new_df['purchase_date_average'] = new_df['purchase_date_diff']/new_df['card_id_size']
    new_df['purchase_date_uptonow'] = (datetime.datetime.today() - new_df['purchase_date_max']).dt.days
    return new_df

In [66]:
aggregate_by_card_id(temp).T

,0
card_id,C_ID_92a2005557
transactions_count,260
city_id_nunique,7
merchant_category_id_nunique,41
merchant_id_nunique,94
state_id_nunique,3
subsector_id_nunique,21
purchase_amount_sum,-165.969
purchase_amount_max,2.25839
purchase_amount_min,-0.739395


In [68]:
hist_trans['category_2'].value_counts()

1.0    17830063
3.0     3911795
5.0     3725915
4.0     2618053
2.0     1026535
Name: category_2, dtype: int64

In [69]:
new_hist_trans['category_2'].value_counts()

1.0    1169987
3.0     289525
5.0     259266
4.0     178590
2.0      65663
Name: category_2, dtype: int64

In [71]:
hist_trans['category_3'].value_counts()

A    15589906
B    11677522
C     1844933
Name: category_3, dtype: int64

In [72]:
hist_trans['category_1'].value_counts()

0    27028332
1     2084029
Name: category_1, dtype: int64

In [73]:
new_hist_trans['category_1'].value_counts()

0    1899935
1      63096
Name: category_1, dtype: int64

In [84]:
temp1 = hist_trans.loc[hist_trans['card_id'].isin(['C_ID_05042ebd55', 'C_ID_5c240d6e3c'])]

In [90]:
temp1.T

,29112043,29112044,29112045,29112046,29112047,29112048,29112358,29112359,29112360
authorized_flag,0,1,1,1,0,1,1,1,1
card_id,C_ID_05042ebd55,C_ID_05042ebd55,C_ID_05042ebd55,C_ID_05042ebd55,C_ID_05042ebd55,C_ID_05042ebd55,C_ID_5c240d6e3c,C_ID_5c240d6e3c,C_ID_5c240d6e3c
city_id,-1,20,-1,-1,-1,-1,3,331,331
category_1,0,0,0,0,0,0,0,0,0
installments,0,0,0,0,0,0,0,0,0
category_3,A,A,A,A,A,A,A,A,A
merchant_category_id,437,422,703,80,80,278,278,514,514
merchant_id,M_ID_ce1e7a7a85,M_ID_0de7fd2bef,M_ID_051eacb2a3,M_ID_9ecff0beda,M_ID_9ecff0beda,M_ID_814793f74b,M_ID_9cdcfe8673,M_ID_1a75f94f92,M_ID_1a75f94f92
month_lag,-4,-1,-2,0,0,0,0,-1,-2
purchase_amount,-0.643976,-0.692948,-0.716855,-0.705585,-0.705134,-0.725871,-0.65774,0.77062,1.13441


In [85]:
temp1[['category_2','category_3', 'category_1', 'authorized_flag']].apply(pd.value_counts)

,category_2,category_3,category_1,authorized_flag
1.0,8.0,NaN,NaN,7.0
3.0,1.0,NaN,NaN,NaN
A,NaN,9.0,NaN,NaN
0,NaN,NaN,9.0,2.0


In [91]:
k =temp1.groupby(['card_id', 'category_2']).agg({"purchase_amount":'sum'})

In [92]:
k

purchase_amount
card_id         category_2                 
C_ID_05042ebd55 1.0               -3.497421
                3.0               -0.692948
C_ID_5c240d6e3c 1.0                1.247291

In [93]:
['_'.join(c) for c in k.columns.values]

['p_u_r_c_h_a_s_e___a_m_o_u_n_t']

In [94]:
k.columns.values

array(['purchase_amount'], dtype=object)

In [95]:
k.columns

Index(['purchase_amount'], dtype='object')

In [148]:
l = temp1.pivot_table(index='card_id',columns='category_2',aggfunc={"purchase_amount": ['sum']})

In [149]:
l.columns = ['cat_2_{0[2]}_{0[0]}_{0[1]}'.format(col) if col[2] else col[2] for col in l.columns.tolist()]

In [150]:
l.reset_index()

,card_id,cat_2_1.0_purchase_amount_sum,cat_2_3.0_purchase_amount_sum
0,C_ID_05042ebd55,-3.497421,-0.692948
1,C_ID_5c240d6e3c,1.247291,NaN


In [200]:
p = temp1.pivot_table(index='card_id',columns='category_2' ,aggfunc={"purchase_amount": ['sum', 'mean']})

In [201]:
p

purchase_amount                              
                           mean                 sum          
category_2                  1.0       3.0       1.0       3.0
card_id                                                      
C_ID_05042ebd55       -0.699484 -0.692948 -3.497421 -0.692948
C_ID_5c240d6e3c        0.415764       NaN  1.247291       NaN

In [202]:
['cat_2_{0[2]}_{0[0]}_{0[1]}'.format(col) if col[2] else col[2] for col in p.columns.tolist()]

['cat_2_1.0_purchase_amount_mean',
 'cat_2_3.0_purchase_amount_mean',
 'cat_2_1.0_purchase_amount_sum',
 'cat_2_3.0_purchase_amount_sum']

In [159]:
aggregate_by_card_id(temp1).T

,0,1
card_id,C_ID_05042ebd55,C_ID_5c240d6e3c
transactions_count,6,3
city_id_nunique,2,2
merchant_category_id_nunique,5,2
merchant_id_nunique,5,2
state_id_nunique,2,1
subsector_id_nunique,4,2
purchase_amount_sum,-4.19037,1.24729
purchase_amount_max,-0.643976,1.13441
purchase_amount_min,-0.725871,-0.65774


In [170]:
temp1.pivot_table(index='card_id',columns='authorized_flag' ,aggfunc={"purchase_amount": ['sum', 'mean']})

purchase_amount                             
                           mean                sum          
authorized_flag               0         1        0         1
card_id                                                     
C_ID_05042ebd55       -0.674555 -0.710314 -1.34911 -2.841258
C_ID_5c240d6e3c             NaN  0.415764      NaN  1.247291

In [172]:
temp1['authorized_flag'].value_counts()

1    7
0    2
Name: authorized_flag, dtype: int64

In [209]:
def agg_on_cat(df, category, feature):
    temp_df = df.pivot_table(index='card_id', columns=category, aggfunc={feature: ['sum', 'mean']})
    cols = [category + '_{0[2]}_{0[0]}_{0[1]}'.format(col) for col in temp_df.columns.tolist()]
    temp_df.columns = cols
    return temp_df

In [210]:
agg_on_cat(temp1, 'category_2', 'purchase_amount')

,category_2_1.0_purchase_amount_mean,category_2_3.0_purchase_amount_mean,category_2_1.0_purchase_amount_sum,category_2_3.0_purchase_amount_sum
card_id,,,,
C_ID_05042ebd55,-0.699484,-0.692948,-3.497421,-0.692948
C_ID_5c240d6e3c,0.415764,NaN,1.247291,NaN


In [211]:
agg_on_cat(temp1, 'category_3', 'purchase_amount')

,category_3_A_purchase_amount_mean,category_3_A_purchase_amount_sum
card_id,,
C_ID_05042ebd55,-0.698395,-4.190368
C_ID_5c240d6e3c,0.415764,1.247291


In [212]:
agg_on_cat(temp1, 'authorized_flag', 'purchase_amount')

,authorized_flag_0_purchase_amount_mean,authorized_flag_1_purchase_amount_mean,authorized_flag_0_purchase_amount_sum,authorized_flag_1_purchase_amount_sum
card_id,,,,
C_ID_05042ebd55,-0.674555,-0.710314,-1.34911,-2.841258
C_ID_5c240d6e3c,NaN,0.415764,NaN,1.247291


In [213]:
new_hist_trans = feather.read_dataframe('new_hist_trans')

In [214]:
agg_on_cat(new_hist_trans, 'category_2', 'purchase_amount')

,category_2_1.0_purchase_amount_mean,category_2_2.0_purchase_amount_mean,category_2_3.0_purchase_amount_mean,category_2_4.0_purchase_amount_mean,category_2_5.0_purchase_amount_mean,category_2_1.0_purchase_amount_sum,category_2_2.0_purchase_amount_sum,category_2_3.0_purchase_amount_sum,category_2_4.0_purchase_amount_sum,category_2_5.0_purchase_amount_sum
card_id,,,,,,,,,,
C_ID_00007093c1,-0.656749,NaN,-0.671775,NaN,NaN,-0.656749,NaN,-0.671775,NaN,NaN
C_ID_0001238066,-0.580966,NaN,NaN,NaN,-0.495945,-13.362220,NaN,NaN,NaN,-1.487835
C_ID_0001506ef0,NaN,NaN,-0.723677,NaN,NaN,NaN,NaN,-1.447354,NaN,NaN
C_ID_0001793786,0.139747,-0.344766,0.102887,NaN,-0.361628,2.375707,-2.758131,0.514433,NaN,-0.361628
C_ID_000183fdda,NaN,NaN,-0.599162,NaN,NaN,NaN,NaN,-6.590778,NaN,NaN
C_ID_0002709b5a,-0.680791,-0.726367,NaN,NaN,-0.611669,-0.680791,-0.726367,NaN,NaN,-0.611669
C_ID_000298032a,-0.641872,NaN,NaN,NaN,NaN,-0.641872,NaN,NaN,NaN,NaN
C_ID_0002ba3c2e,NaN,-0.690353,NaN,NaN,NaN,NaN,-2.071059,NaN,NaN,NaN
C_ID_0002c7c2c1,-0.654908,NaN,NaN,NaN,NaN,-3.929448,NaN,NaN,NaN,NaN


In [216]:
p

purchase_amount                              
                           mean                 sum          
category_2                  1.0       3.0       1.0       3.0
card_id                                                      
C_ID_05042ebd55       -0.699484 -0.692948 -3.497421 -0.692948
C_ID_5c240d6e3c        0.415764       NaN  1.247291       NaN

In [224]:
temp1.groupby(['card_id','category_2'])['purchase_amount'].agg(['sum']).groupby('card_id').sum()

,sum
card_id,
C_ID_05042ebd55,-4.190368
C_ID_5c240d6e3c,1.247291


In [225]:
temp1.groupby('card_id').agg({'category_2': 'count'})

,category_2
card_id,
C_ID_05042ebd55,6
C_ID_5c240d6e3c,3
